# Data curation: MDS part III scores

Form 238v1: MDS-UPDRS Scoring Summary
Form 508v1: MDS-UPDRS Part 3 Scoring Summary

In [1]:
# import packages
import os
import platform # don't need
import pandas as pd
import re
import h5py #save hdf5 but pandas has a function...
import numpy as np

C:\ProgramData\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [73]:
filename = r'//FS2.smpp.local\RTO\CIS-PD MUSC\codelist.sas7bdat'
codelist = pd.read_sas(filename)
codelist.columns = codelist.columns.str.replace('z', '')
codelist = codelist.applymap(lambda x: x.decode() if isinstance(x, bytes) else x)
filename = r'//FS2.smpp.local\RTO\CIS-PD MUSC\datadictionary.sas7bdat'
datadict = pd.read_sas(filename)
datadict.columns = datadict.columns.str.replace('z', '')
#datadict = datadict.applymap(lambda x: x.decode() if isinstance(x, bytes) else x)

In [ ]:
# debug

datadict = datadict.applymap(lambda x: x.decode() if isinstance(x, bytes) else x)

In [65]:
datadict.loc[33]

FormID                                              238
CRFNb                                            b'238'
FieldNb                                          b'Q08'
FieldNm        b'Who is filling out this questionnaire'
SASNm                                               NaN
SASLabel                                            NaN
FieldTypeNm                             b'Radio button'
Length                                                0
Unit                                                NaN
Required                                              2
CodeGroup                                           580
Name: 33, dtype: object

In [74]:
# load decoded forms
filename = r'//FS2.smpp.local\RTO\CIS-PD MUSC\decoded_forms\form238.h5'
form238 = pd.read_hdf(filename)
filename = r'//FS2.smpp.local\RTO\CIS-PD MUSC\decoded_forms\form508.h5'
form508 = pd.read_hdf(filename)
form238 = form238.rename(index=str, columns={'SubjectCode':'Subject ID',
                                  'VisitNm':'Visit'})
form508 = form508.rename(index=str, columns={'SubjectCode':'Subject ID',
                                  'VisitNm':'Visit'})

In [61]:
codelist.head()

,GroupID,ItemNb,ItemNm
0,1.0,1.0,"Global non-project related tables, such as zCo..."
1,1.0,2.0,"Global project related study design tables, su..."
2,1.0,3.0,"Global project related form setup tables, such..."
3,1.0,4.0,"Subject related No CRF tables, such as subject..."
4,1.0,5.0,"CRF related no CRF tables, such as subject CRF..."


In [59]:
datadict.head()

,FormID,CRFNb,FieldNb,FieldNm,SASNm,SASLabel,FieldTypeNm,Length,Unit,Required,CodeGroup
0,101.0,b'101',b'zConsentDate',b'Date of signed informed consent',NaN,NaN,b'Date',0.0,NaN,0.0,NaN
1,101.0,b'101',b'zAge',b'Age',NaN,NaN,b'Number',0.0,NaN,0.0,NaN
2,101.0,b'101',b'Q03',b'Motor fluctuations believed to be on average...,NaN,NaN,b'Radio button',0.0,NaN,2.0,51.0
3,101.0,b'101',b'Q04',b'Diagnosis of idiopathic Parkinson\x92s disease',NaN,NaN,b'Radio button',0.0,NaN,0.0,51.0
4,101.0,b'101',b'Q05',b'Hoehn & Yahr stage 1-3',NaN,NaN,b'Radio button',0.0,NaN,0.0,51.0


In [54]:
datadict.loc[datadict.zFormID == 238.0]

,zFormID,zCRFNb,zFieldNb,zFieldNm,zSASNm,zSASLabel,zFieldTypeNm,zLength,zUnit,zRequired,zCodeGroup
24,238.0,b'238',b'zDataCollected',b'Data Collected',NaN,NaN,b'Radio button',0.0,NaN,1.0,51.0
25,238.0,b'238',b'zFormDate',b'Date of assessment',NaN,NaN,b'Date',0.0,NaN,1.0,NaN
26,238.0,b'238',b'Q01',b'Primary source of information',NaN,NaN,b'Radio button',0.0,NaN,2.0,580.0
27,238.0,b'238',b'Q02',b'1.1 Cognitive Impairment',NaN,NaN,b'Number',0.0,b'(0-4)',2.0,NaN
28,238.0,b'238',b'Q03',b'1.2 Hallucinations and Psychosis',NaN,NaN,b'Number',0.0,b'(0-4)',2.0,NaN
29,238.0,b'238',b'Q04',b'1.3 Depressed Mood',NaN,NaN,b'Number',0.0,b'(0-4)',2.0,NaN
30,238.0,b'238',b'Q05',b'1.4 Anxious Mood',NaN,NaN,b'Number',0.0,b'(0-4)',2.0,NaN
31,238.0,b'238',b'Q06',b'1.5 Apathy',NaN,NaN,b'Number',0.0,b'(0-4)',2.0,NaN
32,238.0,b'238',b'Q07',b'1.6 Features of Dopamine Dysregulation Syndr...,NaN,NaN,b'Number',0.0,b'(0-4)',2.0,NaN
33,238.0,b'238',b'Q08',b'Who is filling out this questionnaire',NaN,NaN,b'Radio button',0.0,NaN,2.0,580.0


In [55]:
datadict.loc[datadict.zFormID == 508.0]

,zFormID,zCRFNb,zFieldNb,zFieldNm,zSASNm,zSASLabel,zFieldTypeNm,zLength,zUnit,zRequired,zCodeGroup
258,508.0,b'508',b'zDataCollected',b'Data Collected',NaN,NaN,b'Radio button',0.0,NaN,1.0,51.0
259,508.0,b'508',b'zFormDate',b'Date of assessment',NaN,NaN,b'Date',0.0,NaN,1.0,NaN
260,508.0,b'508',b'zFormTime',b'Time of assessment',NaN,NaN,b'Date and/or Time',0.0,NaN,4.0,NaN
261,508.0,b'508',b'Q01',b'What is the participant\x92s state',NaN,NaN,b'Radio button',0.0,NaN,0.0,581.0
262,508.0,b'508',b'Q02',b'Hours since last Parkinson\x92s Disease medi...,NaN,NaN,b'Number',0.0,b'hours',4.0,NaN
263,508.0,b'508',b'Q33',b'3.1 Speech',NaN,NaN,b'Number',0.0,b'(0-4)',2.0,NaN
264,508.0,b'508',b'Q34',b'3.2 Facial Expression',NaN,NaN,b'Number',0.0,b'(0-4)',2.0,NaN
265,508.0,b'508',b'Q35',b'3.3 Rigidity - Neck',NaN,NaN,b'Number',0.0,b'(0-4)',2.0,NaN
266,508.0,b'508',b'Q36',b'3.3 Rigidity - Right Upper Extremity',NaN,NaN,b'Number',0.0,b'(0-4)',2.0,NaN
267,508.0,b'508',b'Q37',b'3.3 Rigidity - Left Upper Extremity',NaN,NaN,b'Number',0.0,b'(0-4)',2.0,NaN


# Check form 238 and 508 contents

In [50]:
form238.head(2)

,SubjID,SiteID,Visit,FormDate,Q02,Q03,Q04,Q05,Q06,Q07,...,Q74,DataCollected,Q01,Q08,Q29,Q30,Q31,Q66,Q67,Q68
0,1000.0,1313.0,Baseline,2017-06-15,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,Yes,Patient,Patient,Yes,On,Yes,Yes,No,Bilateral involvement without impairment of ba...
1,1000.0,1313.0,2 Weeks,2017-06-29,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,Yes,Patient,Patient,Yes,On,Yes,No,NaN,Bilateral involvement without impairment of ba...


In [45]:
form508.head(2)

,SubjID,SiteID,Visit,FormDate,Q33,Q34,Q35,Q36,Q37,Q38,...,Q64,Q65,Q02,Q01,Q66,Q67,Q68,DataCollected,Qb2_UTC,FormTime
0,1003.0,1313.0,2 Weeks: Time 0,2017-07-03,2.0,2.0,2.0,2.0,2.0,2.0,...,0.0,1.0,NaN,Off,No,NaN,Severe disability; still able to walk or stand...,Yes,2017-07-03 15:20:44,10:20:44
1,1003.0,1313.0,2 Weeks: Time 60,2017-07-03,1.0,2.0,2.0,2.0,2.0,2.0,...,0.0,0.0,NaN,On,No,NaN,Bilateral involvement without impairment of ba...,Yes,2017-07-03 17:11:39,12:11:39


# Modify forms 508 and 238

In [78]:
# drop 508 columns
form508 = form508.drop(columns=['Q02', 'Q01', 'DataCollected', 'Qb2_UTC', 'FormTime'])

In [75]:
# drop form238 columns
# delete Q32 which is 3.C1 Minutes since last levodopa dose
form238 = form238.drop(columns=['Q02', 'Q03', 'Q04', 'Q05',
'Q06', 'Q07', 'Q09', 'Q10', 'Q11', 'Q12', 'Q13', 'Q14', 'Q15', 'Q16',
'Q17', 'Q18', 'Q19', 'Q20', 'Q21', 'Q22', 'Q23', 'Q24', 'Q25', 'Q26',
'Q27', 'Q28', 'Q32',
'Q69', 'Q70', 'Q71', 'Q72',
'Q73', 'Q74', 'DataCollected', 'Q01', 'Q08', 'Q29', 'Q30', 'Q31'])

In [79]:
form508.columns

Index(['Subject ID', 'SiteID', 'Visit', 'FormDate', 'Q33', 'Q34', 'Q35', 'Q36',
       'Q37', 'Q38', 'Q39', 'Q40', 'Q41', 'Q42', 'Q43', 'Q44', 'Q45', 'Q46',
       'Q47', 'Q48', 'Q49', 'Q50', 'Q51', 'Q52', 'Q53', 'Q54', 'Q55', 'Q56',
       'Q57', 'Q58', 'Q59', 'Q60', 'Q61', 'Q62', 'Q63', 'Q64', 'Q65', 'Q66',
       'Q67', 'Q68'],
      dtype='object')

In [76]:
form238.columns

Index(['Subject ID', 'SiteID', 'Visit', 'FormDate', 'Q33', 'Q34', 'Q35', 'Q36',
       'Q37', 'Q38', 'Q39', 'Q40', 'Q41', 'Q42', 'Q43', 'Q44', 'Q45', 'Q46',
       'Q47', 'Q48', 'Q49', 'Q50', 'Q51', 'Q52', 'Q53', 'Q54', 'Q55', 'Q56',
       'Q57', 'Q58', 'Q59', 'Q60', 'Q61', 'Q62', 'Q63', 'Q64', 'Q65', 'Q66',
       'Q67', 'Q68'],
      dtype='object')

In [81]:
# concatenate forms 508 and 238 dataframes
combo = pd.concat([form238, form508])

In [134]:
# Change column names to question with body part if relevant

# current column names
colname = combo.columns

In [135]:
colname

Index(['Subject ID', 'SiteID', 'Visit', 'FormDate', 'Q33', 'Q34', 'Q35', 'Q36',
       'Q37', 'Q38', 'Q39', 'Q40', 'Q41', 'Q42', 'Q43', 'Q44', 'Q45', 'Q46',
       'Q47', 'Q48', 'Q49', 'Q50', 'Q51', 'Q52', 'Q53', 'Q54', 'Q55', 'Q56',
       'Q57', 'Q58', 'Q59', 'Q60', 'Q61', 'Q62', 'Q63', 'Q64', 'Q65', 'Q66',
       'Q67', 'Q68'],
      dtype='object')

In [90]:
oldname = ['Q33', 'Q34', 'Q35', 'Q36', 'Q37', 'Q38', 'Q39', 'Q40', 'Q41', 
           'Q42', 'Q43', 'Q44', 'Q45', 'Q46', 'Q47', 'Q48', 'Q49', 'Q50', 
           'Q51', 'Q52', 'Q53', 'Q54', 'Q55', 'Q56', 'Q57', 'Q58', 'Q59', 
           'Q60', 'Q61', 'Q62', 'Q63', 'Q64', 'Q65', 'Q66', 'Q67', 'Q68']

In [93]:
# change column names for Q33-68 with 3.13 etc.
newname = [# questions 33-34
'3.1',
'3.2',
# questions 35-39
'3.3 Neck',
'3.3 Right Upper Extremity',
'3.3 Left Upper Extremity',
'3.3 Right Lower Extremity',
'3.3 Left Lower Extremity',
# questions 40-41
'3.4 Right Hand',
'3.4 Left Hand',
# questions 42-43
'3.5 Right Hand',
'3.5 Left Hand',
# questions 44-45
'3.6 Right Hand',
'3.6 Left Hand',
# questions 46-47
'3.7 Right Foot',
'3.7 Left Foot',
# questions 48-49
'3.8 Right Leg',
'3.8 Left Leg',
# questions 50-55
'3.9',
'3.10',
'3.11',
'3.12',
'3.13',
'3.14',
# questions 56-57
'3.15 Right Hand',
'3.15 Left Hand',
# questions 58-59
'3.16 Right Hand',
'3.16 Left Hand',
# questions 60-64
'3.17 Right Upper Extremity',
'3.17 Left Upper Extremity',
'3.17 Right Lower Extremity',
'3.17 Left Lower Extremity',
'3.17 Lip-Jaw',
# question 65
'3.18',
# questions 66-67
'3.19A',
'3.19B',
# question 68
'3.20']

In [137]:
col_dict = dict(zip(oldname,newname))
print(col_dict)

{'Q33': '3.1', 'Q34': '3.2', 'Q35': '3.3 Neck', 'Q36': '3.3 Right Upper Extremity', 'Q37': '3.3 Left Upper Extremity', 'Q38': '3.3 Right Lower Extremity', 'Q39': '3.3 Left Lower Extremity', 'Q40': '3.4 Right Hand', 'Q41': '3.4 Left Hand', 'Q42': '3.5 Right Hand', 'Q43': '3.5 Left Hand', 'Q44': '3.6 Right Hand', 'Q45': '3.6 Left Hand', 'Q46': '3.7 Right Foot', 'Q47': '3.7 Left Foot', 'Q48': '3.8 Right Leg', 'Q49': '3.8 Left Leg', 'Q50': '3.9', 'Q51': '3.10', 'Q52': '3.11', 'Q53': '3.12', 'Q54': '3.13', 'Q55': '3.14', 'Q56': '3.15 Right Hand', 'Q57': '3.15 Left Hand', 'Q58': '3.16 Right Hand', 'Q59': '3.16 Left Hand', 'Q60': '3.17 Right Upper Extremity', 'Q61': '3.17 Left Upper Extremity', 'Q62': '3.17 Right Lower Extremity', 'Q63': '3.17 Left Lower Extremity', 'Q64': '3.17 Lip-Jaw', 'Q65': '3.18', 'Q66': '3.19A', 'Q67': '3.19B', 'Q68': '3.20'}


In [136]:
# change colname using col_dict
#col_dict['Q33']
for i, k in enumerate(colname):
    if i>3:
        k = col_dict.get(k)
#     else:   
#         k

In [126]:
col_dict.get('Q33')

'3.1'

In [150]:
for i, k in enumerate(colname):
    if i>3:
        k = col_dict.get(k)

In [153]:
colname.replace(col_dict)

AttributeError: 'Index' object has no attribute 'replace'

In [151]:
colname

Index(['Subject ID', 'SiteID', 'Visit', 'FormDate', 'Q33', 'Q34', 'Q35', 'Q36',
       'Q37', 'Q38', 'Q39', 'Q40', 'Q41', 'Q42', 'Q43', 'Q44', 'Q45', 'Q46',
       'Q47', 'Q48', 'Q49', 'Q50', 'Q51', 'Q52', 'Q53', 'Q54', 'Q55', 'Q56',
       'Q57', 'Q58', 'Q59', 'Q60', 'Q61', 'Q62', 'Q63', 'Q64', 'Q65', 'Q66',
       'Q67', 'Q68'],
      dtype='object')

In [121]:
colname[4]

'Q33'

In [141]:
print(type(colname),type(newname))

<class 'pandas.core.indexes.base.Index'> <class 'list'>


In [140]:
colname[4:] = newname

TypeError: Index does not support mutable operations

In [138]:
colname

Index(['Subject ID', 'SiteID', 'Visit', 'FormDate', 'Q33', 'Q34', 'Q35', 'Q36',
       'Q37', 'Q38', 'Q39', 'Q40', 'Q41', 'Q42', 'Q43', 'Q44', 'Q45', 'Q46',
       'Q47', 'Q48', 'Q49', 'Q50', 'Q51', 'Q52', 'Q53', 'Q54', 'Q55', 'Q56',
       'Q57', 'Q58', 'Q59', 'Q60', 'Q61', 'Q62', 'Q63', 'Q64', 'Q65', 'Q66',
       'Q67', 'Q68'],
      dtype='object')

In [119]:
colname.apply(lambda x:col_dict.get(x) if col_dict.get(x) is not None else x)

AttributeError: 'Index' object has no attribute 'apply'

In [109]:
type(colname)

pandas.core.indexes.base.Index

In [ ]:
# clean up float



In [ ]:
# save file as updrs_part3.h5
filename = r'//FS2.smpp.local\RTO\CIS-PD MUSC\decoded_forms\updrs_part3.h5'
# save pd.read_hdf(filename)
with open(filename,'wb') as f:
    combo.to_hdf(filename, mode='w')